In [1]:
import matplotlib.pyplot as plt
from datetime import datetime, time
from dateutil.relativedelta import relativedelta
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ta
import mplfinance as mpf
from itertools import product
import numpy as np

In [2]:
from src.DataUpdateModule import DataUpdateModule
from src.Strategy import Strategy
DUM = DataUpdateModule(options_interval_minutes = 5)
strategy = Strategy()

In [3]:
df_stock, df_options = DUM.read_data(datetime(2022,1,7))
strategy.compute_tech_indicators(df_stock)

In [11]:
model_num = 0
interval = 5

# Trade parameters
AUM = 6.5 * 1e6
max_risk = 0.025
commission_dollars = 0.15

In [12]:
enter_long, exit_long, enter_short, exit_short = strategy.entry_exit_signals(model_num = model_num)
params = strategy.get_models(model_num)

In [13]:

max_risk_dollars = AUM * max_risk

# Initiations
position = 0
trades = list()
tradeable_instances = list(df_options.loc[:, "time"].unique())
prev_instance = tradeable_instances[0] - relativedelta(minutes = interval)
for trade_instance in tradeable_instances:
    df_options_instance = df_options[df_options["time"] == trade_instance]

    match position:
        case 0:
            if trade_instance.time() < time(14, 30):
                if enter_long[(enter_long.index < trade_instance)].iloc[-1]:
                    metadata = open_long_trade(df_options_instance, params, max_risk_dollars)
                    if metadata is None:
                        continue # Skip if trade could not be opened
                    else:
                        trades.append(metadata)
                        position = 1
                elif enter_short[(enter_short.index < trade_instance)].iloc[-1]:
                    metadata = open_short_trade(df_options_instance, params, max_risk_dollars)
                    if metadata is None:
                        continue # Skip if trade could not be opened
                    else:
                        trades.append(metadata)
                        position = -1
        case 1:
            latest_trade = trades[-1]
            # exit long criteria
            exit_signal = exit_long[(exit_long.index < trade_instance) & (exit_long.index >= prev_instance)].any()
            # stoploss hit
            stoploss_hit = check_long_stoploss(df_options_instance, latest_trade, params)
            # checks
            if exit_signal or stoploss_hit:
                trades[-1] |= close_long_trade(df_options_instance, latest_trade, reason = "exit_criteria" if exit_signal else "stoploss")
                position = 0
        case -1:    
            latest_trade = trades[-1]
            # exit short criteria
            exit_signal = exit_short[(exit_short.index < trade_instance) & (exit_short.index >= prev_instance)].any()
            # stoploss hit
            stoploss_hit = check_short_stoploss(df_options_instance, latest_trade, params)
            # checks
            if exit_signal or stoploss_hit:
                trades[-1] |= close_short_trade(df_options_instance, latest_trade, reason = "exit_criteria" if exit_signal else "stoploss")
                position = 0
        case _:
            raise ValueError("Invalid position value.")
trades = pd.DataFrame(trades)

In [14]:
trades.pnl.sum()

np.float64(-38943.8)

In [15]:
trades

,entry_time,direction,entry_spot,entry_atm_strike,leg1_strike,leg2_strike,entry_leg1_price,entry_leg2_price,entry_unit_spread,entry_unit_maxloss,contracts,stoploss,exit_time,exit_spot,exit_leg1_price,exit_leg2_price,exit_unit_spread,exit_reason,unit_pnl_gross,pnl
0,2022-01-07 10:05:00-05:00,-1,468.28000000,468.00000000,468.00000000,469.00000000,1.11000000,1.61000000,0.51000000,0.49000000,1658,0.61200000,2022-01-07 10:10:00-05:00,467.48000000,1.02000000,0.60000000,0.42000000,exit_criteria,0.09000000,13927.20000000
1,2022-01-07 10:30:00-05:00,1,467.06000000,467.00000000,467.00000000,466.00000000,1.17000000,0.82000000,0.35000000,0.65000000,1250,0.42000000,2022-01-07 10:35:00-05:00,466.93000000,1.24000000,0.84000000,0.40000000,exit_criteria,-0.05000000,-7000.00000000
2,2022-01-07 11:00:00-05:00,1,465.31000000,465.00000000,465.00000000,464.00000000,1.14000000,0.80000000,0.34000000,0.66000000,1231,0.40800000,2022-01-07 11:05:00-05:00,465.22000000,1.21000000,0.84000000,0.37000000,exit_criteria,-0.03000000,-4431.60000000
3,2022-01-07 11:10:00-05:00,1,465.62000000,466.00000000,466.00000000,465.00000000,1.40000000,0.99000000,0.41000000,0.59000000,1377,0.49200000,2022-01-07 11:15:00-05:00,465.33000000,1.58000000,1.07000000,0.51000000,exit_criteria,-0.10000000,-14596.20000000
4,2022-01-07 12:05:00-05:00,-1,467.28000000,467.00000000,467.00000000,468.00000000,0.87000000,1.37000000,0.50000000,0.50000000,1624,0.60000000,2022-01-07 12:10:00-05:00,467.38000000,1.24000000,0.68000000,0.56000000,exit_criteria,-0.06000000,-10718.40000000
5,2022-01-07 12:15:00-05:00,1,467.67000000,468.00000000,468.00000000,467.00000000,1.12000000,0.71000000,0.41000000,0.59000000,1377,0.49200000,2022-01-07 12:20:00-05:00,467.23000000,1.38000000,0.86000000,0.52000000,exit_criteria,-0.11000000,-15973.20000000
6,2022-01-07 12:40:00-05:00,-1,466.77000000,467.00000000,467.00000000,468.00000000,1.03000000,1.65000000,0.39000000,0.61000000,1331,0.46800000,2022-01-07 12:45:00-05:00,466.86000000,0.85000000,0.42000000,0.43000000,exit_criteria,-0.04000000,-6122.60000000
7,2022-01-07 13:00:00-05:00,1,467.28000000,467.00000000,467.00000000,466.00000000,0.73000000,0.43000000,0.30000000,0.70000000,1160,0.36000000,2022-01-07 13:05:00-05:00,467.24000000,0.74000000,0.42000000,0.32000000,exit_criteria,-0.02000000,-3016.00000000
8,2022-01-07 13:10:00-05:00,1,467.43000000,467.00000000,467.00000000,466.00000000,0.64000000,0.37000000,0.27000000,0.73000000,1113,0.32400000,2022-01-07 13:15:00-05:00,467.48000000,0.61000000,0.34000000,0.27000000,exit_criteria,0.00000000,-667.80000000
9,2022-01-07 13:20:00-05:00,-1,467.36000000,467.00000000,467.00000000,468.00000000,0.64000000,1.16000000,0.49000000,0.51000000,1593,0.58800000,2022-01-07 13:25:00-05:00,467.18000000,0.92000000,0.43000000,0.49000000,exit_criteria,-0.00000000,-955.80000000


## TESTS